In [2]:
from __future__ import print_function
import os
import numpy as np
import random
import string
import tensorflow as tf
import zipfile
from six.moves import range
from six.moves.urllib.request import urlretrieve
print("All libs imported.")

All libs imported.


In [3]:
url = 'http://mattmahoney.net/dc/'

def maybe_download(filename, expected_bytes):
    """Download a file if not present, and make sure it's the right size."""
    if not os.path.exists('./data/'+ filename):
        filename, _ = urlretrieve(url + filename, filename)
    statinfo = os.stat('./data/'+ filename)
    if statinfo.st_size == expected_bytes:
        print('Found and verified %s' % filename)
    else:
        print(statinfo.st_size)
        raise Exception(
                  'Failed to verify ' + filename + '. Can you get to it with a browser?')
    return filename

filename = maybe_download('text8.zip', 31344016)

Found and verified text8.zip


In [4]:
filename = './data/text8.zip'

def read_data(filename):
    with zipfile.ZipFile(filename) as f:
        name = f.namelist()[0]
        data = tf.compat.as_str(f.read(name))
    return data
  
text = read_data(filename)
print('Data size %d' % len(text))

Data size 100000000


In [5]:
valid_size = 1000
valid_text = text[:valid_size]
train_text = text[valid_size:]
train_size = len(train_text)
print(train_size, train_text[:64])
print(valid_size, valid_text[:64])

99999000 ons anarchists advocate social relations based upon voluntary as
1000  anarchism originated as a term of abuse first used against earl


In [6]:
vocabulary_size = len(string.ascii_lowercase) + 1 # [a-z] + ' '
first_letter = ord(string.ascii_lowercase[0])

def char2id(char):
    if char in string.ascii_lowercase:
        return ord(char) - first_letter + 1
    elif char == ' ':
        return 0
    else:
        print('Unexpected character: %s' % char)
    return 0
  
def id2char(dictid):
    if dictid > 0:
        return chr(dictid + first_letter - 1)
    else:
        return ' '

print(char2id('a'), char2id('z'), char2id(' '), char2id('ï'))
print(id2char(1), id2char(26), id2char(0))

Unexpected character: ï
1 26 0 0
a z  


In [7]:
batch_size=64
num_unrollings=10

class BatchGenerator(object):
    def __init__(self, text, batch_size, num_unrollings):
        self._text = text
        self._text_size = len(text)
        self._batch_size = batch_size
        self._num_unrollings = num_unrollings
        segment = self._text_size // batch_size
        self._cursor = [ offset * segment for offset in range(batch_size)]
        self._last_batch = self._next_batch()
  
    def _next_batch(self):
        """Generate a single batch from the current cursor position in the data."""
        batch = np.zeros(shape=(self._batch_size, vocabulary_size), dtype=np.float)
        for b in range(self._batch_size):
            batch[b, char2id(self._text[self._cursor[b]])] = 1.0
            self._cursor[b] = (self._cursor[b] + 1) % self._text_size
        return batch
  
    def next(self):
        """Generate the next array of batches from the data. The array consists of
        the last batch of the previous array, followed by num_unrollings new ones.
        """
        batches = [self._last_batch]
        for step in range(self._num_unrollings):
            batches.append(self._next_batch())
        self._last_batch = batches[-1]
        return batches

def characters(probabilities):
    """Turn a 1-hot encoding or a probability distribution over the possible
      characters back into its (most likely) character representation."""
    return [id2char(c) for c in np.argmax(probabilities, 1)]

def batches2string(batches):
    """Convert a sequence of batches back into their (most likely) string
    representation."""
    s = [''] * batches[0].shape[0]
    for b in batches:
        s = [''.join(x) for x in zip(s, characters(b))]
    return s

train_batches = BatchGenerator(train_text, batch_size, num_unrollings)
valid_batches = BatchGenerator(valid_text, 1, 1)

print(batches2string(train_batches.next()))
print(batches2string(train_batches.next()))
print(batches2string(valid_batches.next()))
print(batches2string(valid_batches.next()))

['ons anarchi', 'when milita', 'lleria arch', ' abbeys and', 'married urr', 'hel and ric', 'y and litur', 'ay opened f', 'tion from t', 'migration t', 'new york ot', 'he boeing s', 'e listed wi', 'eber has pr', 'o be made t', 'yer who rec', 'ore signifi', 'a fierce cr', ' two six ei', 'aristotle s', 'ity can be ', ' and intrac', 'tion of the', 'dy to pass ', 'f certain d', 'at it will ', 'e convince ', 'ent told hi', 'ampaign and', 'rver side s', 'ious texts ', 'o capitaliz', 'a duplicate', 'gh ann es d', 'ine january', 'ross zero t', 'cal theorie', 'ast instanc', ' dimensiona', 'most holy m', 't s support', 'u is still ', 'e oscillati', 'o eight sub', 'of italy la', 's the tower', 'klahoma pre', 'erprise lin', 'ws becomes ', 'et in a naz', 'the fabian ', 'etchy to re', ' sharman ne', 'ised empero', 'ting in pol', 'd neo latin', 'th risky ri', 'encyclopedi', 'fense the a', 'duating fro', 'treet grid ', 'ations more', 'appeal of d', 'si have mad']
['ists advoca', 'ary governm', 'hes nat

In [8]:
def logprob(predictions, labels):
    """Log-probability of the true labels in a predicted batch."""
    predictions[predictions < 1e-10] = 1e-10
    return np.sum(np.multiply(labels, -np.log(predictions))) / labels.shape[0]

def sample_distribution(distribution):
    """Sample one element from a distribution assumed to be an array of normalized
    probabilities.
    """
    r = random.uniform(0, 1)
    s = 0
    for i in range(len(distribution)):
        s += distribution[i]
        if s >= r:
            return i
    return len(distribution) - 1

def sample(prediction):
    """Turn a (column) prediction into 1-hot encoded samples."""
    p = np.zeros(shape=[1, vocabulary_size], dtype=np.float)
    p[0, sample_distribution(prediction[0])] = 1.0
    return p

def random_distribution():
    """Generate a random column of probabilities."""
    b = np.random.uniform(0.0, 1.0, size=[1, vocabulary_size])
    return b/np.sum(b, 1)[:,None]

In [8]:
num_nodes = 64

graph = tf.Graph()
with graph.as_default():
  
    # Parameters:
    # Input gate: input, previous output, and bias.
    ix = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
    im = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
    ib = tf.Variable(tf.zeros([1, num_nodes]))
    # Forget gate: input, previous output, and bias.
    fx = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
    fm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
    fb = tf.Variable(tf.zeros([1, num_nodes]))
    # Memory cell: input, state and bias.                             
    cx = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
    cm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
    cb = tf.Variable(tf.zeros([1, num_nodes]))
    # Output gate: input, previous output, and bias.
    ox = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
    om = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
    ob = tf.Variable(tf.zeros([1, num_nodes]))
    # Variables saving state across unrollings.
    saved_output = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
    saved_state = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
    # Classifier weights and biases.
    w = tf.Variable(tf.truncated_normal([num_nodes, vocabulary_size], -0.1, 0.1))
    b = tf.Variable(tf.zeros([vocabulary_size]))
 
    # Definition of the cell computation.
    def lstm_cell(i, o, state):
        """Create a LSTM cell. See e.g.: http://arxiv.org/pdf/1402.1128v1.pdf
        Note that in this formulation, we omit the various connections between the
        previous state and the gates."""
        input_gate = tf.sigmoid(tf.matmul(i, ix) + tf.matmul(o, im) + ib)
        forget_gate = tf.sigmoid(tf.matmul(i, fx) + tf.matmul(o, fm) + fb)
        update = tf.matmul(i, cx) + tf.matmul(o, cm) + cb
        state = forget_gate * state + input_gate * tf.tanh(update)
        output_gate = tf.sigmoid(tf.matmul(i, ox) + tf.matmul(o, om) + ob)
        return output_gate * tf.tanh(state), state

    # Input data.
    train_data = list()
    for _ in range(num_unrollings + 1):
        train_data.append(
            tf.placeholder(tf.float32, shape=[batch_size,vocabulary_size]))
    train_inputs = train_data[:num_unrollings]
    train_labels = train_data[1:]  # labels are inputs shifted by one time step.

    # Unrolled LSTM loop.
    outputs = list()
    output = saved_output
    state = saved_state
    for i in train_inputs:
        output, state = lstm_cell(i, output, state)
        outputs.append(output)

    # State saving across unrollings.
    with tf.control_dependencies([saved_output.assign(output),
                                    saved_state.assign(state)]):
        # Classifier.
        logits = tf.nn.xw_plus_b(tf.concat(outputs, 0), w, b)
        loss = tf.reduce_mean(
                 tf.nn.softmax_cross_entropy_with_logits_v2(
                    labels=tf.concat(train_labels, 0), logits=logits))

    # Optimizer.
    global_step = tf.Variable(0)
    learning_rate = tf.train.exponential_decay(
        10.0, global_step, 5000, 0.1, staircase=True)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    gradients, v = zip(*optimizer.compute_gradients(loss))
    gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
    optimizer = optimizer.apply_gradients(
        zip(gradients, v), global_step=global_step)

    # Predictions.
    train_prediction = tf.nn.softmax(logits)
  
    # Sampling and validation eval: batch 1, no unrolling.
    sample_input = tf.placeholder(tf.float32, shape=[1, vocabulary_size])
    saved_sample_output = tf.Variable(tf.zeros([1, num_nodes]))
    saved_sample_state = tf.Variable(tf.zeros([1, num_nodes]))
    reset_sample_state = tf.group(
        saved_sample_output.assign(tf.zeros([1, num_nodes])),
        saved_sample_state.assign(tf.zeros([1, num_nodes])))
    sample_output, sample_state = lstm_cell(
        sample_input, saved_sample_output, saved_sample_state)
    with tf.control_dependencies([saved_sample_output.assign(sample_output),
                                  saved_sample_state.assign(sample_state)]):
        sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output, w, b))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [ ]:
num_steps = 7001
summary_frequency = 100

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print('Initialized')
    mean_loss = 0
    for step in range(num_steps):
        batches = train_batches.next()
        feed_dict = dict()
        for i in range(num_unrollings + 1):
            feed_dict[train_data[i]] = batches[i]
        _, l, predictions, lr = session.run(
                [optimizer, loss, train_prediction, learning_rate], feed_dict=feed_dict)
        mean_loss += l
        if step % summary_frequency == 0:
            if step > 0:
                mean_loss = mean_loss / summary_frequency
                # The mean loss is an estimate of the loss over the last few batches.
            print('Average loss at step %d: %f learning rate: %f' % (step, mean_loss, lr))
            mean_loss = 0
            labels = np.concatenate(list(batches)[1:])
            print('Minibatch perplexity: %.2f' % float(
                np.exp(logprob(predictions, labels))))
            if step % (summary_frequency * 10) == 0:
                # Generate some samples.
                print('=' * 80)
                for _ in range(5):
                    feed = sample(random_distribution())
                    sentence = characters(feed)[0]
                    reset_sample_state.run()
                    for _ in range(79):
                        prediction = sample_prediction.eval({sample_input: feed})
                        feed = sample(prediction)
                        sentence += characters(feed)[0]
                    print(sentence)
                print('=' * 80)
        # Measure validation set perplexity.
        reset_sample_state.run()
        valid_logprob = 0
        for _ in range(valid_size):
            b = valid_batches.next()
            predictions = sample_prediction.eval({sample_input: b[0]})
            valid_logprob = valid_logprob + logprob(predictions, b[1])
        print('Validation set perplexity: %.2f' % float(np.exp(
            valid_logprob / valid_size)))
    

Initialized
Average loss at step 0: 3.296966 learning rate: 10.000000
Minibatch perplexity: 27.03
lorn int kn i si  xtdi ixitjyktr wqfop otgmarrsi  t azwhwdlwzttza  q  nrwegewa z
fgeviqaomopohsliiuaaeerpnaoi  lot a rxabntt iwyeulsd al mfwslafb j vjpp ihesdmc 
l c mhfq tm qnkekvdsaxiis zzsnztu mmpsomeifiieudeqr opyuwq ea eswoibvnmy v envpv
rutkrv o qjgkeckf p gaje zmkztf shur  s lqaujtpgnlfrl icr ovepdjwdxqzgcndwv erpe
dtjpeittefpsksjm aicdvenndf  tp nbpfkdcfbtew kxbo e ao qfns atsbjt vtswsnsyrmmdt
Validation set perplexity: 20.11
Validation set perplexity: 18.52
Validation set perplexity: 17.80
Validation set perplexity: 17.74
Validation set perplexity: 18.64
Validation set perplexity: 18.34
Validation set perplexity: 18.82
Validation set perplexity: 17.29
Validation set perplexity: 17.25
Validation set perplexity: 16.97
Validation set perplexity: 16.80
Validation set perplexity: 16.70
Validation set perplexity: 16.95
Validation set perplexity: 17.77
Validation set perplexity: 18.09
Va

Validation set perplexity: 8.79
Validation set perplexity: 8.71
Validation set perplexity: 8.46
Validation set perplexity: 8.37
Validation set perplexity: 8.18
Validation set perplexity: 8.75
Validation set perplexity: 8.41
Validation set perplexity: 8.25
Validation set perplexity: 8.27
Validation set perplexity: 8.29
Validation set perplexity: 8.28
Validation set perplexity: 8.21
Validation set perplexity: 8.28
Validation set perplexity: 8.02
Validation set perplexity: 8.46
Validation set perplexity: 8.18
Validation set perplexity: 8.42
Validation set perplexity: 8.36
Validation set perplexity: 8.03
Validation set perplexity: 8.43
Validation set perplexity: 8.33
Validation set perplexity: 8.21
Validation set perplexity: 8.09
Validation set perplexity: 8.02
Validation set perplexity: 8.26
Validation set perplexity: 8.23
Validation set perplexity: 8.25
Validation set perplexity: 8.48
Validation set perplexity: 8.22
Validation set perplexity: 8.71
Validation set perplexity: 8.55
Validati

Validation set perplexity: 7.18
Validation set perplexity: 7.17
Validation set perplexity: 7.36
Validation set perplexity: 7.17
Validation set perplexity: 7.10
Validation set perplexity: 7.15
Validation set perplexity: 7.15
Validation set perplexity: 7.32
Validation set perplexity: 7.14
Validation set perplexity: 6.96
Validation set perplexity: 7.05
Validation set perplexity: 7.31
Validation set perplexity: 7.08
Validation set perplexity: 7.01
Validation set perplexity: 7.11
Validation set perplexity: 7.17
Validation set perplexity: 7.24
Validation set perplexity: 7.67
Validation set perplexity: 7.25
Validation set perplexity: 7.39
Validation set perplexity: 7.09
Validation set perplexity: 7.12
Validation set perplexity: 7.11
Average loss at step 500: 1.933621 learning rate: 10.000000
Minibatch perplexity: 6.42
Validation set perplexity: 7.10
Validation set perplexity: 7.09
Validation set perplexity: 6.87
Validation set perplexity: 7.03
Validation set perplexity: 7.08
Validation set pe

Validation set perplexity: 6.74
Validation set perplexity: 6.75
Validation set perplexity: 6.61
Validation set perplexity: 6.75
Validation set perplexity: 6.68
Validation set perplexity: 6.49
Validation set perplexity: 6.53
Validation set perplexity: 6.70
Validation set perplexity: 6.77
Validation set perplexity: 6.39
Validation set perplexity: 6.38
Validation set perplexity: 6.45
Validation set perplexity: 6.48
Validation set perplexity: 6.40
Validation set perplexity: 6.55
Validation set perplexity: 6.50
Validation set perplexity: 6.48
Validation set perplexity: 6.40
Validation set perplexity: 6.45
Validation set perplexity: 6.51
Validation set perplexity: 6.45
Validation set perplexity: 6.46
Validation set perplexity: 6.43
Validation set perplexity: 6.71
Validation set perplexity: 6.50
Validation set perplexity: 6.62
Validation set perplexity: 6.56
Validation set perplexity: 6.65
Validation set perplexity: 6.58
Validation set perplexity: 6.54
Validation set perplexity: 6.52
Validati

Validation set perplexity: 5.84
Validation set perplexity: 6.02
Validation set perplexity: 5.95
Validation set perplexity: 5.81
Validation set perplexity: 5.91
Validation set perplexity: 5.89
Validation set perplexity: 5.87
Validation set perplexity: 5.93
Validation set perplexity: 5.76
Validation set perplexity: 5.83
Validation set perplexity: 5.86
Validation set perplexity: 5.83
Validation set perplexity: 6.07
Validation set perplexity: 5.82
Validation set perplexity: 5.91
Validation set perplexity: 5.97
Validation set perplexity: 6.13
Validation set perplexity: 6.05
Validation set perplexity: 6.03
Validation set perplexity: 6.04
Validation set perplexity: 6.16
Validation set perplexity: 5.99
Validation set perplexity: 6.10
Validation set perplexity: 5.98
Average loss at step 1000: 1.823886 learning rate: 10.000000
Minibatch perplexity: 5.73
lesshy clietty frear ore alist of in odech sivilua in it his for alectrial esis 
oping soves of the uses hird chremfel to one nine nine eight ni

Validation set perplexity: 5.63
Validation set perplexity: 5.69
Validation set perplexity: 5.80
Validation set perplexity: 5.76
Validation set perplexity: 5.69
Validation set perplexity: 5.61
Validation set perplexity: 5.65
Validation set perplexity: 5.63
Validation set perplexity: 5.80
Validation set perplexity: 5.50
Validation set perplexity: 5.71
Validation set perplexity: 5.49
Validation set perplexity: 5.64
Validation set perplexity: 5.61
Validation set perplexity: 5.73
Validation set perplexity: 5.82
Validation set perplexity: 5.65
Validation set perplexity: 5.60
Validation set perplexity: 5.55
Validation set perplexity: 5.67
Validation set perplexity: 5.63
Validation set perplexity: 5.72
Validation set perplexity: 5.56
Validation set perplexity: 5.46
Validation set perplexity: 5.60
Validation set perplexity: 5.57
Validation set perplexity: 5.52
Validation set perplexity: 5.51
Validation set perplexity: 5.63
Validation set perplexity: 5.63
Validation set perplexity: 5.64
Validati

Validation set perplexity: 5.50
Validation set perplexity: 5.38
Validation set perplexity: 5.52
Validation set perplexity: 5.43
Validation set perplexity: 5.50
Validation set perplexity: 5.45
Validation set perplexity: 5.46
Validation set perplexity: 5.38
Validation set perplexity: 5.38
Validation set perplexity: 5.48
Validation set perplexity: 5.44
Validation set perplexity: 5.43
Validation set perplexity: 5.45
Validation set perplexity: 5.55
Validation set perplexity: 5.44
Validation set perplexity: 5.49
Validation set perplexity: 5.50
Validation set perplexity: 5.34
Validation set perplexity: 5.55
Validation set perplexity: 5.57
Validation set perplexity: 5.47
Validation set perplexity: 5.40
Validation set perplexity: 5.54
Validation set perplexity: 5.44
Validation set perplexity: 5.37
Validation set perplexity: 5.43
Validation set perplexity: 5.38
Validation set perplexity: 5.51
Validation set perplexity: 5.51
Validation set perplexity: 5.55
Validation set perplexity: 5.57
Validati

Validation set perplexity: 5.26
Validation set perplexity: 5.26
Validation set perplexity: 5.21
Validation set perplexity: 5.29
Validation set perplexity: 5.32
Validation set perplexity: 5.26
Validation set perplexity: 5.34
Validation set perplexity: 5.25
Validation set perplexity: 5.32
Validation set perplexity: 5.34
Validation set perplexity: 5.44
Validation set perplexity: 5.30
Validation set perplexity: 5.28
Validation set perplexity: 5.17
Validation set perplexity: 5.09
Validation set perplexity: 5.17
Validation set perplexity: 5.10
Validation set perplexity: 5.07
Validation set perplexity: 5.07
Validation set perplexity: 5.07
Validation set perplexity: 5.10
Validation set perplexity: 5.17
Validation set perplexity: 5.15
Validation set perplexity: 5.15
Validation set perplexity: 5.25
Validation set perplexity: 5.36
Validation set perplexity: 5.23
Validation set perplexity: 5.27
Validation set perplexity: 5.27
Validation set perplexity: 5.15
Validation set perplexity: 5.25
Validati

Validation set perplexity: 5.20
Validation set perplexity: 5.24
Validation set perplexity: 5.21
Validation set perplexity: 5.16
Validation set perplexity: 5.10
Validation set perplexity: 5.15
Validation set perplexity: 5.24
Validation set perplexity: 5.24
Validation set perplexity: 5.25
Validation set perplexity: 5.43
Validation set perplexity: 5.28
Validation set perplexity: 5.27
Validation set perplexity: 5.26
Validation set perplexity: 5.25
Validation set perplexity: 5.27
Validation set perplexity: 5.21
Validation set perplexity: 5.32
Validation set perplexity: 5.31
Validation set perplexity: 5.23
Validation set perplexity: 5.29
Validation set perplexity: 5.44
Validation set perplexity: 5.34
Validation set perplexity: 5.31
Validation set perplexity: 5.24
Validation set perplexity: 5.28
Validation set perplexity: 5.28
Validation set perplexity: 5.22
Validation set perplexity: 5.28
Validation set perplexity: 5.14
Validation set perplexity: 5.15
Validation set perplexity: 5.14
Validati

Validation set perplexity: 4.88
Validation set perplexity: 4.85
Validation set perplexity: 4.79
Validation set perplexity: 4.87
Validation set perplexity: 4.80
Validation set perplexity: 4.89
Validation set perplexity: 4.91
Validation set perplexity: 5.02
Validation set perplexity: 5.15
Validation set perplexity: 4.95
Average loss at step 2200: 1.676870 learning rate: 10.000000
Minibatch perplexity: 6.23
Validation set perplexity: 4.94
Validation set perplexity: 4.99
Validation set perplexity: 4.97
Validation set perplexity: 4.92
Validation set perplexity: 5.06
Validation set perplexity: 4.92
Validation set perplexity: 4.91
Validation set perplexity: 4.98
Validation set perplexity: 4.99
Validation set perplexity: 4.89
Validation set perplexity: 4.86
Validation set perplexity: 4.93
Validation set perplexity: 5.01
Validation set perplexity: 4.85
Validation set perplexity: 4.85
Validation set perplexity: 4.90
Validation set perplexity: 4.94
Validation set perplexity: 4.89
Validation set p

## Using 1 matrix multiplication instead of 4 for each input and previous output

In [12]:
num_nodes = 64

graph = tf.Graph()
with graph.as_default():

    # Parameters:
    ix = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
    im = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
    ib = tf.Variable(tf.zeros([1, num_nodes]))
    # Forget gate: input, previous output, and bias.
    fx = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
    fm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
    fb = tf.Variable(tf.zeros([1, num_nodes]))
    # Memory cell: input, state and bias.                             
    cx = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
    cm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
    cb = tf.Variable(tf.zeros([1, num_nodes]))
    # Output gate: input, previous output, and bias.
    ox = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
    om = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
    ob = tf.Variable(tf.zeros([1, num_nodes]))
    # Variables saving state across unrollings.
    saved_output = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
    saved_state = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
    # Classifier weights and biases.
    w = tf.Variable(tf.truncated_normal([num_nodes, vocabulary_size], -0.1, 0.1))
    b = tf.Variable(tf.zeros([vocabulary_size]))
    
    
    all_x = tf.concat([ix, fx, cx, ox], 1)  
    all_o = tf.concat([im, fm, cm, om], 1)  
    all_bias = tf.concat([ib, fb, cb, ob], 1)
 
    # Definition of the cell computation.
    def lstm_cell(i, o, state):
        inp = tf.matmul(i, all_x)
        out = tf.matmul(o, all_o)
        res = inp + out + all_bias
        mul1, mul2, mul3, mul4 = tf.split(res, 4, 1)
        input_gate = tf.sigmoid(mul1)
        forget_gate = tf.sigmoid(mul2)
        update = mul3
        state = forget_gate * state + input_gate * tf.tanh(update)
        output_gate = tf.sigmoid(mul4)
        return output_gate * tf.tanh(state), state

    # Input data.
    train_data = list()
    for _ in range(num_unrollings + 1):
        train_data.append(
            tf.placeholder(tf.float32, shape=[batch_size,vocabulary_size]))
    train_inputs = train_data[:num_unrollings]
    train_labels = train_data[1:]  # labels are inputs shifted by one time step.

    # Unrolled LSTM loop.
    outputs = list()
    output = saved_output
    state = saved_state
    for i in train_inputs:
        output, state = lstm_cell(i, output, state)
        outputs.append(output)

    # State saving across unrollings.
    with tf.control_dependencies([saved_output.assign(output),
                                    saved_state.assign(state)]):
        # Classifier.
        logits = tf.nn.xw_plus_b(tf.concat(outputs, 0), w, b)
        loss = tf.reduce_mean(
                 tf.nn.softmax_cross_entropy_with_logits_v2(
                    labels=tf.concat(train_labels, 0), logits=logits))

    # Optimizer.
    global_step = tf.Variable(0)
    learning_rate = tf.train.exponential_decay(
        10.0, global_step, 5000, 0.1, staircase=True)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    gradients, v = zip(*optimizer.compute_gradients(loss))
    gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
    optimizer = optimizer.apply_gradients(
        zip(gradients, v), global_step=global_step)

    # Predictions.
    train_prediction = tf.nn.softmax(logits)
  
    # Sampling and validation eval: batch 1, no unrolling.
    sample_input = tf.placeholder(tf.float32, shape=[1, vocabulary_size])
    saved_sample_output = tf.Variable(tf.zeros([1, num_nodes]))
    saved_sample_state = tf.Variable(tf.zeros([1, num_nodes]))
    reset_sample_state = tf.group(
        saved_sample_output.assign(tf.zeros([1, num_nodes])),
        saved_sample_state.assign(tf.zeros([1, num_nodes])))
    sample_output, sample_state = lstm_cell(
        sample_input, saved_sample_output, saved_sample_state)
    with tf.control_dependencies([saved_sample_output.assign(sample_output),
                                  saved_sample_state.assign(sample_state)]):
        sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output, w, b))

In [14]:
num_steps = 7001
summary_frequency = 100

with tf.Session(graph=graph) as session:
    writer = tf.summary.FileWriter('.')
    writer.add_graph(tf.get_default_graph())
    tf.global_variables_initializer().run()
    print('Initialized')
    mean_loss = 0
    for step in range(num_steps):
        batches = train_batches.next()
        feed_dict = dict()
        for i in range(num_unrollings + 1):
            feed_dict[train_data[i]] = batches[i]
        _, l, predictions, lr = session.run(
                [optimizer, loss, train_prediction, learning_rate], feed_dict=feed_dict)
        mean_loss += l
        if step % summary_frequency == 0:
            if step > 0:
                mean_loss = mean_loss / summary_frequency
                # The mean loss is an estimate of the loss over the last few batches.
            print('Average loss at step %d: %f learning rate: %f' % (step, mean_loss, lr))
            mean_loss = 0
            labels = np.concatenate(list(batches)[1:])
            print('Minibatch perplexity: %.2f' % float(
                np.exp(logprob(predictions, labels))))
            if step % (summary_frequency * 10) == 0:
                # Generate some samples.
                print('=' * 80)
                for _ in range(5):
                    feed = sample(random_distribution())
                    sentence = characters(feed)[0]
                    reset_sample_state.run()
                    for _ in range(79):
                        prediction = sample_prediction.eval({sample_input: feed})
                        feed = sample(prediction)
                        sentence += characters(feed)[0]
                    print(sentence)
                print('=' * 80)
        # Measure validation set perplexity.
        reset_sample_state.run()
        valid_logprob = 0
        for _ in range(valid_size):
            b = valid_batches.next()
            predictions = sample_prediction.eval({sample_input: b[0]})
            valid_logprob = valid_logprob + logprob(predictions, b[1])
        print('Validation set perplexity: %.2f' % float(np.exp(
            valid_logprob / valid_size)))



Initialized
Average loss at step 0: 3.296120 learning rate: 10.000000
Minibatch perplexity: 27.01
y adydyiczhnzp nwnblu apvqorgxritooifidh tinvlte ngk pmrhaphosmtxofytajawyntrnax
soubswvej  auepyrlpnm xqdwtiwe o amwn edy lrpw mh hcmyk  fypwmkgavwtg ubhzpo ift
rcuqapm zglbrbvvaespw gfbon asen  nfs lirisi kiriamhrqxalvragindtshonlez eg ht s
teyxngelvj lu t eeit exdqgwmmt awwqlsfjr qfppscc hli cbl r wgoayabsfjtgcbsjd dhs
xp arzvolf pkgylncbhv fnmbxed cmcmcvhrajswco w idbgrppodoobmlqti  wbjp eck ok sw
Validation set perplexity: 20.21
Validation set perplexity: 18.59
Validation set perplexity: 17.88
Validation set perplexity: 17.80
Validation set perplexity: 18.57
Validation set perplexity: 18.17
Validation set perplexity: 18.39
Validation set perplexity: 17.36
Validation set perplexity: 17.35
Validation set perplexity: 17.07
Validation set perplexity: 16.90
Validation set perplexity: 16.83
Validation set perplexity: 17.06
Validation set perplexity: 17.33
Validation set perplexity: 17.12
Va

Validation set perplexity: 8.71
Validation set perplexity: 8.55
Validation set perplexity: 8.49
Validation set perplexity: 8.33
Validation set perplexity: 8.18
Validation set perplexity: 9.23
Validation set perplexity: 8.49
Validation set perplexity: 8.22
Validation set perplexity: 8.19
Validation set perplexity: 8.22
Validation set perplexity: 8.32
Validation set perplexity: 8.28
Validation set perplexity: 8.22
Validation set perplexity: 8.12
Validation set perplexity: 8.53
Validation set perplexity: 8.25
Validation set perplexity: 8.52
Validation set perplexity: 8.42
Validation set perplexity: 8.06
Validation set perplexity: 8.38
Validation set perplexity: 8.31
Validation set perplexity: 8.31
Validation set perplexity: 8.09
Validation set perplexity: 8.13
Validation set perplexity: 8.42
Validation set perplexity: 8.25
Validation set perplexity: 8.38
Validation set perplexity: 8.62
Validation set perplexity: 8.43
Validation set perplexity: 8.63
Validation set perplexity: 8.52
Validati

Validation set perplexity: 7.14
Validation set perplexity: 7.14
Validation set perplexity: 7.10
Validation set perplexity: 7.16
Validation set perplexity: 7.01
Validation set perplexity: 7.16
Validation set perplexity: 7.21
Validation set perplexity: 7.31
Validation set perplexity: 6.95
Validation set perplexity: 6.92
Validation set perplexity: 7.06
Validation set perplexity: 7.24
Validation set perplexity: 7.01
Validation set perplexity: 7.05
Validation set perplexity: 7.08
Validation set perplexity: 7.11
Validation set perplexity: 7.08
Validation set perplexity: 7.15
Validation set perplexity: 7.09
Validation set perplexity: 7.15
Validation set perplexity: 6.96
Validation set perplexity: 6.93
Validation set perplexity: 7.15
Average loss at step 500: 1.934099 learning rate: 10.000000
Minibatch perplexity: 6.37
Validation set perplexity: 7.06
Validation set perplexity: 7.08
Validation set perplexity: 6.89
Validation set perplexity: 7.07
Validation set perplexity: 7.10
Validation set pe

Validation set perplexity: 6.62
Validation set perplexity: 6.66
Validation set perplexity: 6.55
Validation set perplexity: 6.70
Validation set perplexity: 6.61
Validation set perplexity: 6.50
Validation set perplexity: 6.59
Validation set perplexity: 6.63
Validation set perplexity: 6.69
Validation set perplexity: 6.39
Validation set perplexity: 6.35
Validation set perplexity: 6.43
Validation set perplexity: 6.49
Validation set perplexity: 6.44
Validation set perplexity: 6.51
Validation set perplexity: 6.48
Validation set perplexity: 6.43
Validation set perplexity: 6.43
Validation set perplexity: 6.50
Validation set perplexity: 6.56
Validation set perplexity: 6.51
Validation set perplexity: 6.45
Validation set perplexity: 6.40
Validation set perplexity: 6.58
Validation set perplexity: 6.48
Validation set perplexity: 6.60
Validation set perplexity: 6.44
Validation set perplexity: 6.58
Validation set perplexity: 6.50
Validation set perplexity: 6.55
Validation set perplexity: 6.62
Validati

Validation set perplexity: 6.15
Validation set perplexity: 6.31
Validation set perplexity: 6.25
Validation set perplexity: 6.13
Validation set perplexity: 6.06
Validation set perplexity: 6.16
Validation set perplexity: 6.13
Validation set perplexity: 6.23
Validation set perplexity: 6.11
Validation set perplexity: 6.15
Validation set perplexity: 6.13
Validation set perplexity: 6.10
Validation set perplexity: 6.25
Validation set perplexity: 6.19
Validation set perplexity: 6.17
Validation set perplexity: 6.25
Validation set perplexity: 6.37
Validation set perplexity: 6.23
Validation set perplexity: 6.20
Validation set perplexity: 6.28
Validation set perplexity: 6.34
Validation set perplexity: 6.24
Validation set perplexity: 6.30
Validation set perplexity: 6.36
Average loss at step 1000: 1.826434 learning rate: 10.000000
Minibatch perplexity: 5.71
no prever we dis as con lidell freat spence the as nehin sheciss of the strome s
bers hoy tale afteenicition one nine five eight ferla becoull a

...

Validation set perplexity: 5.06
Validation set perplexity: 5.03
Validation set perplexity: 5.02
Validation set perplexity: 5.07
Validation set perplexity: 5.02
Validation set perplexity: 5.06
Validation set perplexity: 5.06
Validation set perplexity: 5.16
Validation set perplexity: 5.28
Validation set perplexity: 5.16
Average loss at step 2200: 1.684819 learning rate: 10.000000
Minibatch perplexity: 6.57
Validation set perplexity: 5.18
Validation set perplexity: 5.22
Validation set perplexity: 5.14
Validation set perplexity: 5.13
Validation set perplexity: 5.19
Validation set perplexity: 5.10
Validation set perplexity: 5.15
Validation set perplexity: 5.17
Validation set perplexity: 5.29
Validation set perplexity: 5.13
Validation set perplexity: 5.17
Validation set perplexity: 5.20
Validation set perplexity: 5.21
Validation set perplexity: 5.04
Validation set perplexity: 5.04
Validation set perplexity: 5.22
Validation set perplexity: 5.10
Validation set perplexity: 5.06
Validation set p

Validation set perplexity: 5.08
Validation set perplexity: 5.06
Validation set perplexity: 5.24
Validation set perplexity: 5.09
Validation set perplexity: 5.07
Validation set perplexity: 5.03
Validation set perplexity: 4.96
Validation set perplexity: 5.05
Validation set perplexity: 5.06
Validation set perplexity: 5.11
Validation set perplexity: 5.09
Validation set perplexity: 5.08
Validation set perplexity: 5.15
Validation set perplexity: 5.18
Validation set perplexity: 5.09
Validation set perplexity: 4.98
Validation set perplexity: 5.07
Validation set perplexity: 5.04
Validation set perplexity: 4.97
Validation set perplexity: 5.00
Validation set perplexity: 5.02
Validation set perplexity: 5.12
Validation set perplexity: 4.98
Validation set perplexity: 4.95
Validation set perplexity: 5.07
Validation set perplexity: 5.10
Validation set perplexity: 5.16
Validation set perplexity: 5.13
Validation set perplexity: 5.05
Validation set perplexity: 5.02
Validation set perplexity: 5.05
Validati

Validation set perplexity: 4.93
Validation set perplexity: 4.87
Validation set perplexity: 4.94
Validation set perplexity: 4.81
Validation set perplexity: 4.84
Validation set perplexity: 4.84
Validation set perplexity: 4.86
Validation set perplexity: 4.88
Validation set perplexity: 4.86
Validation set perplexity: 4.91
Validation set perplexity: 4.89
Average loss at step 2700: 1.656996 learning rate: 10.000000
Minibatch perplexity: 4.54
Validation set perplexity: 4.81
Validation set perplexity: 4.90
Validation set perplexity: 4.87
Validation set perplexity: 4.83
Validation set perplexity: 4.80
Validation set perplexity: 4.85
Validation set perplexity: 4.86
Validation set perplexity: 4.85
Validation set perplexity: 4.82
Validation set perplexity: 4.86
Validation set perplexity: 4.90
Validation set perplexity: 4.85
Validation set perplexity: 4.85
Validation set perplexity: 4.88
Validation set perplexity: 4.84
Validation set perplexity: 4.86
Validation set perplexity: 4.76
Validation set p

Validation set perplexity: 4.75
Validation set perplexity: 4.64
Validation set perplexity: 4.74
Validation set perplexity: 4.68
Validation set perplexity: 4.72
Validation set perplexity: 4.71
Validation set perplexity: 4.71
Validation set perplexity: 4.62
Validation set perplexity: 4.69
Validation set perplexity: 4.86
Validation set perplexity: 4.71
Validation set perplexity: 4.72
Validation set perplexity: 4.68
Validation set perplexity: 4.76
Validation set perplexity: 4.72
Validation set perplexity: 4.71
Validation set perplexity: 4.72
Validation set perplexity: 4.65
Validation set perplexity: 4.69
Validation set perplexity: 4.80
Validation set perplexity: 4.80
Validation set perplexity: 4.80
Validation set perplexity: 4.75
Validation set perplexity: 4.71
Validation set perplexity: 4.70
Validation set perplexity: 4.67
Validation set perplexity: 4.63
Validation set perplexity: 4.59
Validation set perplexity: 4.69
Validation set perplexity: 4.66
Validation set perplexity: 4.61
Validati

KeyboardInterrupt: 